# 2.1 LangChain Inputs and Outputs

## 🎯 Learning Objectives

In this notebook, you'll learn the **fundamental ways to interact with LLMs** in LangChain:

1. **Single Invocation** - Using `.invoke()` for one-off requests
2. **Batch Processing** - Using `.batch()` and `.generate()` for multiple requests
3. **Message Types** - Building conversations with `SystemMessage`, `HumanMessage`, and `AIMessage`
4. **Response Handling** - Extracting content from `AIMessage` and `LLMResult` objects

## 📚 Prerequisites

- Basic Python knowledge
- API keys configured (OpenAI, Groq, or Databricks)
- Completed: `1. Getting Started` notebooks

## 🔑 Key Concepts

| Method | Input | Output | Use Case |
|--------|-------|--------|----------|
| `.invoke()` | String or Messages | AIMessage | Single request |
| `.batch()` | List of Strings | List of AIMessages | Multiple simple requests |
| `.generate()` | List of Message Lists | LLMResult | Multiple conversations with metadata |

---

In [9]:
# ============================================================================
# ENVIRONMENT SETUP: Load API Keys & Import Dependencies
# ============================================================================
# We use python-dotenv to securely load API keys from a .env file
# This is a best practice - never hardcode API keys in your notebooks!
# ============================================================================

from dotenv import load_dotenv
import os
import sys
import platform

# Load environment variables from .env file
load_dotenv()

# Add parent directory to path for importing helpers
sys.path.append(os.path.abspath("../.."))

# Import our LLM factory functions
# - get_groq_llm(): Creates a Groq-hosted LLM (fast inference with open-source models)
# - get_openai_llm(): Creates an OpenAI GPT model
# - get_databricks_llm(): Creates a Databricks-hosted LLM
from helpers.utils import get_groq_llm, get_openai_llm, get_databricks_llm

print("✅ Environment variables loaded successfully!")
print(f"📍 Running on: {platform.system()}")

# -----------------------------------------------------------------------------
# Initialize the LLM based on platform or preference
# The choice of LLM affects tool calling capabilities and speed
# -----------------------------------------------------------------------------
if sys.platform == "win32":
    # Windows: Use Groq for fast inference
    llm = get_groq_llm()
elif sys.platform == "darwin":
    # macOS: Use Databricks-hosted Gemini
    llm = get_databricks_llm("databricks-gemini-2-5-pro")  
else:
    # Linux: Default to Groq
    llm = get_groq_llm()

# Print which LLM we're using
if hasattr(llm, 'model_name'):
    print(f"🤖 LLM initialized: {llm.model_name}")
elif hasattr(llm, 'model'):
    print(f"🤖 LLM initialized: {llm.model}")
else:
    print("🤖 LLM initialized successfully")

✅ Environment variables loaded successfully!
📍 Running on: Darwin
🤖 LLM initialized: databricks-gemini-2-5-pro


In [12]:
# ============================================================================
# BASIC LLM INVOCATION: The .invoke() Method
# ============================================================================
# The .invoke() method is the simplest way to interact with an LLM
# - Input: A string prompt (or list of messages)
# - Output: An AIMessage object containing the response
# ============================================================================

result = llm.invoke("Tell me a joke")

# The response is an AIMessage object - use .content to get the text
print("Response type:", type(result))
print("Response content:", result.content)

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field'

## 📦 Batch Processing with LLMs

When you need to process **multiple prompts efficiently**, LangChain provides two approaches:

1. **`.batch()`** - Simple and clean, accepts list of strings
2. **`.generate()`** - More control, expects list of message lists

Batch processing is more efficient than calling `.invoke()` multiple times because:
- Reduces API overhead
- Some providers optimize batch requests
- Easier to manage multiple outputs

In [24]:
# ============================================================================
# METHOD 1: Using .batch() - The Simple Approach
# ============================================================================
# .batch() accepts a list of string prompts and returns a list of AIMessage objects
# This is the recommended approach for simple batch processing
# ============================================================================

prompts = [
    "Tell me a joke about cows",
    "Tell me a joke about parrots"
]

# Process multiple prompts in one call
batch_results = llm.batch(prompts)

# Extract and display each response
print("=" * 60)
print("BATCH PROCESSING RESULTS")
print("=" * 60)
for i, result in enumerate(batch_results):
    print(f"\n📝 Prompt {i+1}: {prompts[i]}")
    print(f"🤖 Response: {result.content}")
    print("-" * 60)

BATCH PROCESSING RESULTS

📝 Prompt 1: Tell me a joke about cows
🤖 Response: Why do cows wear bells?

...Because their horns don't work
------------------------------------------------------------

📝 Prompt 2: Tell me a joke about parrots
🤖 Response: Here's a classic for you:

A woman gets a new parrot, but it has a terrible attitude and an even worse vocabulary. Every other word is a swear word. She tries everything to clean up the bird's language, but nothing works.

Finally, in a fit of rage, she grabs the parrot and shoves it into the freezer.

For a minute, she hears squawking, kicking, and screaming. Then, suddenly, it's completely silent. Horrified that she might have hurt the bird, she quickly opens the freezer door.

The parrot calmly steps out, shivers, and says, "I must apologize for my offensive language and behavior. I promise to improve my vocabulary and conduct myself as a proper gentleman from now on."

The woman is astonished. But before she can ask what caused the sudd

In [25]:
# ============================================================================
# METHOD 2: Using .generate() - More Control & Metadata
# ============================================================================
# .generate() is more powerful but requires structured input:
# - Input: List of message lists (each inner list is a conversation)
# - Output: LLMResult object with generations, token usage, and metadata
# 
# Use .generate() when you need:
# - Access to token usage statistics
# - Full metadata about each generation
# - Complex multi-turn conversations in batch
# ============================================================================

from langchain_core.messages import HumanMessage

# Each inner list represents a separate conversation/request
generate_result = llm.generate([
    [HumanMessage(content="Tell me a joke about cows")],
    [HumanMessage(content="Tell me a joke about parrots")]
])

# The result contains detailed information about each generation
print("Type of result:", type(generate_result))
print("\nNumber of generations:", len(generate_result.generations))

Type of result: <class 'langchain_core.outputs.llm_result.LLMResult'>

Number of generations: 2


In [26]:
# ============================================================================
# OPTIONAL: Inspect the full LLMResult object
# ============================================================================
# The generate_result variable contains the complete LLMResult object
# with all generations and metadata. Uncomment to explore its structure.
# ============================================================================

# Uncomment to see the full structure:
generate_result.generations  # List of ChatGeneration objects

[[ChatGeneration(text='Of course!\n\nWhere do cows go for entertainment?\n\n...To the **moo**-vies', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Of course!\n\nWhere do cows go for entertainment?\n\n...To the **moo**-vies', additional_kwargs={}, response_metadata={'usage': {'prompt_tokens': 6, 'completion_tokens': 22, 'total_tokens': 1577}, 'prompt_tokens': 6, 'completion_tokens': 22, 'total_tokens': 1577, 'model': 'gemini-2.5-pro', 'model_name': 'gemini-2.5-pro', 'finish_reason': 'stop'}, id='lc_run--019c0f53-11ba-79b1-a238-70b27ef353b5-0', tool_calls=[], invalid_tool_calls=[]))],
 [ChatGeneration(text='Of course! Here\'s a classic for you:\n\nA woman goes into a pet shop and sees a beautiful, colorful parrot. She asks the owner, "How much for this magnificent bird?"\n\nThe owner sighs. "He\'s $50, but I have to warn you, he used to live in a brothel. He says some very inappropriate things."\n\nThe woman thinks about it and decides she can handle it. "I\'ll ta

## 💬 Building Conversations with Message Types

LangChain provides three core message types for building conversations:

| Message Type | Purpose | Example |
|--------------|---------|---------|
| **SystemMessage** | Sets the AI's behavior/personality | "You are a helpful restaurant assistant" |
| **HumanMessage** | User's input/questions | "What's on the menu?" |
| **AIMessage** | AI's previous responses | "We have pasta, pizza, and salads" |

**Why use message types?**
- Gives the LLM context about who said what
- Allows multi-turn conversations with history
- System messages guide the AI's behavior throughout the conversation

In [30]:
# ============================================================================
# SWITCHING LLM PROVIDER: Using OpenAI's ChatGPT
# ============================================================================
# LangChain makes it easy to switch between different LLM providers
# Here we switch to OpenAI's GPT-4o-mini for the conversation examples
# The API is consistent - all LLMs support .invoke(), .batch(), .generate()
# ============================================================================

from langchain_openai import ChatOpenAI

# Initialize OpenAI's GPT-4o-mini model
# Note: Requires OPENAI_API_KEY in your .env file
llm = get_databricks_llm("databricks-gpt-5-1")  
print(f"✅ Switched to: {llm.model}")

✅ Switched to: databricks-gpt-5-1


In [31]:
# ============================================================================
# QUICK TEST: Verify the new LLM is working
# ============================================================================
# This is a simple test to confirm the OpenAI model is properly configured
# Note: This is similar to our earlier example but uses a different provider
# ============================================================================

# Simple invocation test with OpenAI
test_result = llm.invoke("Tell me a short joke")
print("✅ OpenAI Response:", test_result.content)

✅ OpenAI Response: Why don’t programmers like nature?

It has too many bugs.


In [32]:
# ============================================================================
# CREATING A MULTI-TURN CONVERSATION
# ============================================================================
# This example simulates a restaurant chatbot conversation
# We build the conversation history manually to show how the LLM maintains context
# ============================================================================

from langchain_core.messages import HumanMessage,SystemMessage,AIMessage

# Build a conversation with history
# The order matters: System → Human → AI → Human → ...
messages = [
    # 1. System message: Sets the assistant's role and personality
    SystemMessage(content="You are a helpful assistant specialized in providing information about BellaVista Italian Restaurant."),
    
    # 2. First user question
    HumanMessage(content="What's on the menu?"),
    
    # 3. AI's previous response (simulated history)
    AIMessage(content="BellaVista offers a variety of Italian dishes including pasta, pizza, and seafood."),
    
    # 4. Follow-up question from user
    HumanMessage(content="Do you have vegan options?")
]

# Display the conversation structure
print("📋 Conversation Structure:")
print("-" * 50)
for msg in messages:
    role = type(msg).__name__.replace("Message", "")
    print(f"{role:>8}: {msg.content[:60]}{'...' if len(msg.content) > 60 else ''}")

📋 Conversation Structure:
--------------------------------------------------
  System: You are a helpful assistant specialized in providing informa...
   Human: What's on the menu?
      AI: BellaVista offers a variety of Italian dishes including past...
   Human: Do you have vegan options?


In [35]:
# ============================================================================
# INSPECTING MESSAGE OBJECTS
# ============================================================================
# Each message is a Pydantic object with:
# - content: The actual text
# - additional_kwargs: Extra parameters (like function calls)
# - response_metadata: Metadata about the response (for AI messages)
# ============================================================================

# View the raw message objects (useful for debugging)
print("📦 Raw Message Objects:")
for i, msg in enumerate(messages):
    print(f"\n[{i}] {type(msg).__name__}:")
    print(f"    content: {msg.content[:200]}...")

📦 Raw Message Objects:

[0] SystemMessage:
    content: You are a helpful assistant specialized in providing information about BellaVista Italian Restaurant....

[1] HumanMessage:
    content: What's on the menu?...

[2] AIMessage:
    content: BellaVista offers a variety of Italian dishes including pasta, pizza, and seafood....

[3] HumanMessage:
    content: Do you have vegan options?...


In [38]:
# ============================================================================
# INVOKING LLM WITH CONVERSATION HISTORY
# ============================================================================
# When you pass a list of messages, the LLM:
# 1. Reads the SystemMessage to understand its role
# 2. Reviews the conversation history (Human/AI exchanges)
# 3. Generates a contextually appropriate response
# ============================================================================

# The LLM will respond to the last HumanMessage with full context
llm_result = llm.invoke(input=messages)

print("🤖 AI Response to 'Do you have vegan options?':")
print("-" * 50)
print(llm_result.content)
print("-" * 50)
print(f"\n📊 Token Usage: {llm_result.response_metadata}")

🤖 AI Response to 'Do you have vegan options?':
--------------------------------------------------
Yes, BellaVista does offer vegan options. While the exact items can vary by day or season, you can typically expect:

- **Salads** – Garden or mixed green salads that can be made vegan by omitting cheese and choosing an oil‑based dressing (like vinaigrette).
- **Bruschetta / Antipasti** – Tomato bruschetta without cheese, grilled or marinated vegetables, olives.
- **Pasta** – Dried pasta (often egg‑free) with:
  - Tomato/basil (pomodoro) sauce  
  - Arrabbiata (spicy tomato)  
  - Aglio e olio (garlic, olive oil, chili)  
  Ask to confirm the pasta is egg‑free and that no butter or cheese is added.
- **Pizza** – Vegetable pizzas that can be made:
  - Without cheese, or  
  - With vegan cheese (if available at your location)  
  Toppings like mushrooms, peppers, onions, olives, artichokes, arugula, etc.
- **Sides** – Roasted potatoes, grilled vegetables, sautéed greens prepared in olive oil

## 🌍 Batch Processing with Conversations

You can also batch process **entire conversations** (not just simple prompts). This is useful for:
- **Parallel translations** - Same content, different system prompts
- **A/B testing prompts** - Compare different phrasings
- **Multi-language support** - Translate to multiple languages simultaneously

The `.generate()` method is particularly useful here as it provides detailed metadata for each generation.

In [39]:
# ============================================================================
# EXAMPLE: Parallel Translation with Different System Prompts
# ============================================================================
# Here we translate the same phrase to multiple languages simultaneously
# Each inner list is a complete conversation with its own SystemMessage
# ============================================================================

# Create two separate conversations with different translation instructions
batch_messages = [
    # Conversation 1: English → German
    [
        SystemMessage(content="You are a helpful assistant that translates English to German. Only respond with the translation, nothing else."),
        HumanMessage(content="Do you have vegan options?")
    ],
    # Conversation 2: English → Spanish
    [
        SystemMessage(content="You are a helpful assistant that translates English to Spanish. Only respond with the translation, nothing else."),
        HumanMessage(content="Do you have vegan options?")
    ],
]

# Generate responses for both conversations in parallel
batch_result = llm.generate(batch_messages)

# Display results
print("🌍 Parallel Translation Results:")
print("=" * 50)
print(f"🇬🇧 Original: 'Do you have vegan options?'")
print(f"🇩🇪 German:   {batch_result.generations[0][0].text}")
print(f"🇪🇸 Spanish:  {batch_result.generations[1][0].text}")
print("=" * 50)

🌍 Parallel Translation Results:
🇬🇧 Original: 'Do you have vegan options?'
🇩🇪 German:   Haben Sie vegane Optionen?
🇪🇸 Spanish:  ¿Tienen opciones veganas?


## 🔍 Extracting Data from LLMResult

The `.generate()` method returns an `LLMResult` object with rich metadata:
- **generations**: List of generated responses (one per input conversation)
- **llm_output**: Aggregate info like total token usage
- **model_dump()**: Convert to dictionary for easy inspection

> **Coming Up:** In later notebooks, we'll explore **Output Parsers** that automatically structure LLM outputs into Python objects!


In [59]:
# ============================================================================
# INSPECTING LLMResult STRUCTURE
# ============================================================================
# The .model_dump() method converts the Pydantic object to a dictionary
# This is useful for:
# - Debugging and understanding the response structure
# - Logging responses to files/databases
# - Extracting specific metadata
# ============================================================================

# Convert to dictionary for inspection
result_dict = batch_result.model_dump()

# Show the structure (keys at top level)
print("📦 LLMResult Structure:")
print("-" * 40)
for key in result_dict.keys():
    print(f"  • {key}")

# Show token usage summary
if 'generations' in result_dict and result_dict['generations']:
    token_usage = result_dict['generations'][0][0]['message']['response_metadata'].get('usage', {})
    print(f"\n📊 Token Usage Summary:")
    print(f"  • Prompt tokens:     {token_usage.get('prompt_tokens', 'N/A')}")
    print(f"  • Completion tokens: {token_usage.get('completion_tokens', 'N/A')}")
    print(f"  • Total tokens:      {token_usage.get('total_tokens', 'N/A')}")

📦 LLMResult Structure:
----------------------------------------
  • generations
  • llm_output
  • run
  • type

📊 Token Usage Summary:
  • Prompt tokens:     36
  • Completion tokens: 16
  • Total tokens:      52


In [60]:
# ============================================================================
# EXTRACTING RESPONSES FROM LLMResult
# ============================================================================
# The generations attribute is a nested list:
# - Outer list: One entry per input conversation
# - Inner list: Multiple generations if n > 1 (default is 1)
# - Each generation has .text for the content
# ============================================================================

# Extract just the text from each generation
translations = [generation[0].text for generation in batch_result.generations]

# Display final results
print("✅ Extracted Translations:")
languages = ["German", "Spanish"]
for lang, translation in zip(languages, translations):
    print(f"  {lang}: {translation}")

# ============================================================================
# 📝 KEY TAKEAWAYS FROM THIS NOTEBOOK:
# ============================================================================
# 1. .invoke() - Single prompt/conversation, returns AIMessage
# 2. .batch()  - Multiple prompts, returns list of AIMessages
# 3. .generate() - Multiple conversations with full metadata (LLMResult)
# 4. Message Types: SystemMessage, HumanMessage, AIMessage for conversations
# 5. LangChain provides a unified API across different LLM providers
# ============================================================================

✅ Extracted Translations:
  German: Haben Sie vegane Optionen?
  Spanish: ¿Tienen opciones veganas?
